In [14]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [16]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [17]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [18]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import *

def initialize_swarm(world : BaseWorld):
    swarm = [SARAgent() for i in range(SWARM_SIZE)]
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)


In [19]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [20]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

comms_protocol = SARCommunicationProtocol()
action_interpreter = SARActionInterpreter()

In [21]:
custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter)

In [22]:
custom_gym.reset(42)

({1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
          [0., 0., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 1., 1., 1.]]),
   'Energy': 100.0},
  2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 1., 1., 0.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 1.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 0., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.]]),
   'Energy': 100.0},
  3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 0., 1., 1., 0., 1., 1.]]),
   'Energy': 100.0},
  4: {'Victims': array([[0., 0., 1., 0., 1., 0., 

### Testing

In [23]:
parallel_api_test(custom_gym, num_cycles=1_000_000)
custom_gym.reset()

Passed Parallel API test


({1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
          [0., 0., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 1., 1., 1.]]),
   'Energy': 100.0},
  2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 1., 1., 0.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 1.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 0., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.]]),
   'Energy': 100.0},
  3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 0., 1., 1., 0., 1., 1.]]),
   'Energy': 100.0},
  4: {'Victims': array([[0., 0., 1., 0., 1., 0., 

# Training

In [24]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet

In [25]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             policy_net= PolicyNet(1, 7, 12), 
             target_net=  PolicyNet(1, 7, 12))

In [26]:
train_loop(custom_gym, model, games=10, seed=42)

Training on thesis.
{1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 1., 1., 0., 1., 0.],
       [0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 1., 0., 1., 1., 1.]]), 'Energy': 100.0}, 2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 0.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1., 0.]]), 'Energy': 100.0}, 3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
       [0., 1., 1., 0., 1., 1., 1.],
       [1., 0., 0., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 1.],
       [1., 1., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0., 1., 1.]]), 'Energy': 100.0}, 4: {'Victims': array([[0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 1., 1., 0.],
       [

c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Model has been saved.
{1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 1., 1., 0., 1., 0.],
       [0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 1., 0., 1., 1., 1.]]), 'Energy': 100.0}, 2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 0.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1., 0.]]), 'Energy': 100.0}, 3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
       [0., 1., 1., 0., 1., 1., 1.],
       [1., 0., 0., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 1.],
       [1., 1., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0., 1., 1.]]), 'Energy': 100.0}, 4: {'Victims': array([[0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 1., 1., 0.],
      

AttributeError: 'dict' object has no attribute 'shape'